In [1]:
# Library import

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from ftplib import FTP_TLS
import tkinter as tk
from tkinter import messagebox

delta=dt.timedelta(days=1)

In [2]:
# Función para asignar los días de la semana a cada fecha, si es festivo se trata como un domingo
import holidays
co_holidays = holidays.Colombia()

def typedays(row,tipo):

     if tipo=='WeekDay':
          return row['Fecha'].weekday()
     
     elif tipo=='WeekMonth':
          return (row['Fecha'].day - 1) // 7 + 1
     
     elif tipo=='DayType':
          if row['Fecha'] in co_holidays:
               return 3
          elif row['Fecha'].weekday()==5:
               return 2
          elif row['Fecha'].weekday()==6:
               return 3
          else:
               return 1

In [3]:
# Get main path and other folders
s_mainpath=Path.cwd()
print(s_mainpath)
s_parentpath=s_mainpath.parent.parent.parent
print(s_parentpath)
# folderpath=s_parentpath.joinpath(r'DatosXM\Despacho\\')

# Ruta General
sRutaDesp=r'C:\Información XM\\Publico\\'

sRutaPrint=s_mainpath.joinpath('Print')

c:\Alejo\Eje de Planeación\Análisis Eléctrico\ScriptsPython\Demanda
c:\Alejo\Eje de Planeación


In [4]:
# Rango de análisis

# s_FechaIni='2022-11-19'
# s_FechaFin='2024-11-18'

s_FechaIni='2025-01-01'
s_FechaFin='2025-12-31'

# tipod='day_w'
tipod='day_osf'

#Camniar a formato fecha
d_FechaIni=dt.datetime.strptime(s_FechaIni,'%Y-%m-%d')
d_FechaFin=dt.datetime.strptime(s_FechaFin,'%Y-%m-%d')

# Create a DataFrame of dates
df_dates = pd.DataFrame(pd.date_range(start=s_FechaIni, end=s_FechaFin, freq='D')  , columns=["Date"])

df_dates['Mes']=df_dates['Date'].dt.month

df_dates

,Date,Mes
0,2025-01-01,1
1,2025-01-02,1
2,2025-01-03,1
3,2025-01-04,1
4,2025-01-05,1
...,...,...
360,2025-12-27,12
361,2025-12-28,12
362,2025-12-29,12
363,2025-12-30,12


In [5]:
# Función para descargar el archivo
def DownFile(fecha_dt,UsuXM,PwsXM,Tipo):

    # Connect to the FTP server (replace with your actual details)
    ftps  = FTP_TLS()
    ftps .connect('xmftps.xm.com.co', 210)  # default port is 210

    # Secure the control connection
    ftps .auth()
    ftps .prot_p()  # Switch to secure data connection (important!)

    ftps .login(UsuXM, PwsXM)

    # Obtener mes y día de la fecha inicial
    # Transformar string en fecha
    # fecha = dt.datetime.strptime(fecha_dt, "%d/%m/%Y")

    # Obtener mes y día
    year= fecha_dt.year
    mes = fecha_dt.month
    dia = fecha_dt.day

    if Tipo=='DemDespacho':
        # Navigate to the directory you want to access
        ftps.cwd(rf"/INFORMACION_XM/Publico/DESPACHO/{year:04d}-{mes:02d}")
    elif Tipo=='Demanda':
        # Calcular el lunes asociado a la semana de la fecha dada
        lunes_asociado = fecha_dt - dt.timedelta(days=fecha_dt.weekday())
        # Obtener mes y día
        year= lunes_asociado.year
        mes = lunes_asociado.month
        dia = lunes_asociado.day
        ftps.cwd(rf"/INFORMACION_XM/PUBLICO/DEMANDAS/Pronostico Oficial/{year:04d}-{mes:02d}")
    else:
        messagebox.showinfo('Estado del proceso',f'No se reconoce el formato {Tipo}', parent=root)
        df=pd.DataFrame()
        return df


    # List files
    files = ftps.nlst()
    # print("Available files:", files)


    if Tipo=='DemDespacho':
        # Download condiciones iniciales de planta
        pathfile=rf"C:\Información XM\Publico\DESPACHO\{year:04d}-{mes:02d}"
        if not os.path.exists(pathfile):
            os.makedirs(pathfile)
        filename = rf"dDEM{mes:02d}{dia:02d}.TXT"    
    elif Tipo=='Demanda':
        # Calcular el lunes asociado a la semana de la fecha dada
        lunes_asociado = fecha_dt - dt.timedelta(days=fecha_dt.weekday())
        # Obtener mes y día
        year= lunes_asociado.year
        mes = lunes_asociado.month
        dia = lunes_asociado.day

        pathfile=rf"C:\Información XM\PUBLICO\DEMANDAS\Pronostico Oficial\{year:04d}-{mes:02d}"
        if not os.path.exists(pathfile):
            os.makedirs(pathfile)
        filename = rf"PRON_AREAS{mes:02d}{dia:02d}.TXT"

    try:
        # print(pathfile + "\\" + filename)
        with open(pathfile + "\\" + filename, 'wb') as f:
            ftps.retrbinary(f"RETR {filename}", f.write)

        ftps.quit()
        # print(f"{filename} downloaded successfully.")


    except:

        df=pd.DataFrame()

    
    return 

In [6]:
folderpath=Path(sRutaDesp + 'Despacho\\')

filetype='dDEM'

l_colDem=['Subarea']

l_SubCar=['ATLANTIC','BOLIVAR','GCM','CORDOSUC','CERROMAT','SubArea Atlantico','SubArea Bolivar','SubArea Cerromatoso','SubArea Cordoba_Sucre','SubArea GCM'] # Lista con las subareas Caribe

df_Factor=pd.DataFrame()
df_Factor_typo_d=pd.DataFrame()

for i in range(1,25): 
    l_colDem.append(i)


for mes in range(1,13):

    df_Dem=pd.DataFrame()
    df_datesAx=df_dates[(df_dates.Mes==mes)]
    df_datesAx=df_datesAx.sort_values(by='Date', ascending=True)

    # print(mes)

    for ind in df_datesAx.index:

        dFecha=df_datesAx.at[ind,'Date']

        ano=dFecha.year
        mes=dFecha.month
        dia=dFecha.day

        UsuXM='1060588666'
        PwsXM='Alejo230709*+'
        Tipo='DemDespacho'
        # DownFile(dFecha,UsuXM,PwsXM,Tipo)

        
        file = filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
        s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
        df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colDem,encoding="ISO-8859-1")
        df_Aux['Fecha']=dFecha
        df_Aux = df_Aux[~df_Aux['Subarea'].isin(['Total','ECUADOR138','ECUADOR220','COROZO','CUATRIC','SubArea Venezuela_Corozo','SubArea Venezuela_Cuatricentenario','SubArea Ecuador138','SubArea Ecuador230'])]

        df_Aux['Subarea'] = df_Aux['Subarea'].replace({
            'SubArea Atlantico': 'ATLANTIC',
            'SubArea Bolivar': 'BOLIVAR',
            'SubArea Cerromatoso': 'CERROMAT',
            'SubArea Cordoba_Sucre': 'CORDOSUC',
            'SubArea GCM': 'GCM'
        })

        df_Dem=pd.concat([df_Dem,df_Aux])


    # df_Dem.to_csv(sRutaPrint.joinpath('D1.csv'))

    # Definir tipo de día
    df_Dem['day_osf']=df_Dem.apply(lambda row: typedays(row,tipo='DayType'),axis=1)
    # Definir día de la semana
    df_Dem['day_w']=df_Dem.apply(lambda row: typedays(row,tipo='WeekDay'),axis=1)
    # Definir semana del mes
    # df_Dem['week_m']=df_Dem.apply(lambda row: typedays(row,tipo='WeekMonth'),axis=1)


    # Definir los periodos en una columna
    df_Dem = pd.melt(df_Dem, id_vars=['Fecha','day_osf','day_w','Subarea'], var_name='Periodo', value_name='DemMW')
    # # df_dataP['Precio']=df_dataP['Precio']/1000

    df_Dem['Mes']=df_Dem['Fecha'].dt.month
    df_Dem['Dia']=df_Dem['Fecha'].dt.day

    # Calcualr energía del mes con todos los valores
    EnerMes=df_Dem.groupby(['Mes'])[['DemMW']].sum().values[0][0]

    # Cálculo de factores para distribuir por día
    df_EneDiaFecha=df_Dem.groupby(['Fecha','Mes','day_osf','day_w'])[['DemMW']].sum()
    df_EneDiaFecha['FactorDay']=df_EneDiaFecha['DemMW']/EnerMes

    # Energía por periodo
    # df_EneDiaFecha.to_csv(sRutaPrint.joinpath('D2.csv'))

    df_EneDia=df_EneDiaFecha.groupby(['Mes','day_osf','day_w'])[['FactorDay']].mean()
    df_EneDia=df_EneDia.reset_index()

    df_FactorAux=df_EneDia[['Mes','day_osf','day_w','FactorDay']]


    # Cálculo de factores para distribuir por hora
    # Cálculo de promedio día
    df_EneDiaFecha=df_Dem.groupby(['Fecha','Mes','day_osf','day_w'])[['DemMW']].sum()
    df_EneDiaM=df_EneDiaFecha.reset_index()
    df_EneDiaM=df_EneDiaM.rename(columns={'DemMW':'DemMWDia'})

    # Cálculo de promedio hora
    df_EnePer=df_Dem.groupby(['Fecha','Mes','day_osf','day_w','Periodo'])[['DemMW']].sum()
    df_EnePer=df_EnePer.reset_index()

    df_EnePer=df_EnePer.merge(df_EneDiaM,left_on=['Fecha','Mes','day_osf','day_w'],right_on=['Fecha','Mes','day_osf','day_w'], how='left')[['Fecha','Mes','day_osf','day_w','Periodo','DemMW','DemMWDia']]

    df_EnePer['FactorHour']=df_EnePer['DemMW']/df_EnePer['DemMWDia']

    # Energía por periodo
    # df_EnePer.to_csv(sRutaPrint.joinpath('D3.csv'))

    df_EnePer=df_EnePer.groupby(['Mes','day_osf','day_w','Periodo'])[['FactorHour']].mean()
    df_EnePer=df_EnePer.reset_index()

    df_EnePer=df_EnePer[['Mes','day_osf','day_w','Periodo','FactorHour']]

    df_FactorAux=df_EnePer.merge(df_FactorAux,left_on=['Mes','day_osf','day_w'],right_on=['Mes','day_osf','day_w'], how='left')[['Mes','day_osf','day_w','Periodo','FactorHour','FactorDay']]

    # Cálculo de los factores para distribuir la demanda en cada subárea de caribe
    df_Aux=df_Dem.groupby(['Fecha','Mes','day_osf','day_w','Periodo'])[['DemMW']].sum()
    df_Aux=df_Aux.reset_index()
    df_Aux=df_Aux.rename(columns={'DemMW':'DemMWDia'})

    # Cálculo de promedio hora
    df_Aux2=df_Dem[df_Dem['Subarea'].isin(l_SubCar)]
    df_Aux2=df_Aux2.groupby(['Fecha','Mes','day_osf','day_w','Subarea','Periodo'])[['DemMW']].sum()
    df_Aux2=df_Aux2.reset_index()


    df_Aux2=df_Aux2.merge(df_Aux,left_on=['Fecha','Mes','day_osf','day_w','Periodo'],right_on=['Fecha','Mes','day_osf','day_w','Periodo'], how='left')[['Fecha','Mes','day_osf','day_w','Subarea','Periodo','DemMW','DemMWDia']]
    df_Aux2['FactorH']=(df_Aux2['DemMW']/df_Aux2['DemMWDia'])

    # df_Aux2.to_csv(sRutaPrint.joinpath('D4.csv'))

    df_Aux2=df_Aux2.groupby(['Mes','day_osf','day_w','Subarea','Periodo'])[['FactorH']].mean()
    df_Aux2=df_Aux2.reset_index()
    df_Aux2=df_Aux2[['Mes','day_osf','day_w','Subarea','Periodo','FactorH']]

    

    df_FacCar=df_Aux2.groupby(['Mes','day_osf','day_w','Periodo'])[['FactorH']].sum()
    df_FacCar=df_FacCar.reset_index()
    df_FacCar=df_FacCar.rename(columns={'FactorH':'F_Car'})
    
    df_FactorAux=df_FactorAux.merge(df_FacCar,left_on=['Mes','day_osf','day_w','Periodo'],right_on=['Mes','day_osf','day_w','Periodo'], how='left')[['Mes','day_osf','day_w','Periodo','FactorDay','FactorHour','F_Car']]

    df_Aux2['Subarea']='F_' + df_Aux2['Subarea']
    df_Aux2 = df_Aux2.pivot(index=['Mes','day_osf','day_w','Periodo'], columns='Subarea', values='FactorH')
    df_Aux2=df_Aux2.reset_index()
    df_FactorAux=df_FactorAux.merge(df_Aux2,left_on=['Mes','day_osf','day_w','Periodo'],right_on=['Mes','day_osf','day_w','Periodo'], how='left')[['Mes'
                                        ,'day_osf','day_w','Periodo','FactorDay','FactorHour','F_Car','F_ATLANTIC', 'F_BOLIVAR','F_CERROMAT', 'F_CORDOSUC', 'F_GCM']]
    df_Factor=pd.concat([df_Factor,df_FactorAux],axis=0)

if tipod=='day_osf':
    # Factores por tipo de día
    df_Factor_typo_d=df_Factor.groupby(['Mes','day_osf','Periodo'])[['FactorDay','FactorHour','F_Car','F_ATLANTIC', 'F_BOLIVAR','F_CERROMAT', 'F_CORDOSUC', 'F_GCM']].mean().reset_index()
else:
    # Factores por día de la semana
    df_Factor_typo_d=df_Factor.groupby(['Mes','day_w','Periodo'])[['FactorDay','FactorHour','F_Car','F_ATLANTIC', 'F_BOLIVAR','F_CERROMAT', 'F_CORDOSUC', 'F_GCM']].mean().reset_index()


df_Factor_typo_d['F_Interior'] = 1 - df_Factor_typo_d['F_Car']
df_Factor_typo_d['F_Caribe2'] = df_Factor_typo_d['F_ATLANTIC'] + df_Factor_typo_d['F_BOLIVAR'] + df_Factor_typo_d['F_GCM']

# df_FacDay=df_Factor_typo_d.groupby(['Mes','day_w'])[['F_Interior','F_Car','F_Caribe2']].mean()
# df_FacDay=df_FacDay.reset_index()



# df_EneDia.to_csv('demMEs.csv')
# df_FacCar=df_FacCar.reset_index()
# df_FacCar[(df_FacCar.day_w==5)]

# df_Factor_typo_d.to_csv(sRutaPrint.joinpath('Factores.csv'))
# df_FacDay.to_csv(sRutaPrint.joinpath('Factores.csv'),index=False)



In [7]:
# Lectura de datos de enrtrada para el cálculo
s_parentpath=Path(r'C:\Alejo\Eje de Planeación\Análisis Energético\Ejecuciones 30 días\2026-01\Semana_Tri_MAM')
filepath=s_parentpath.joinpath(s_parentpath,'DemandaUPME.xlsx')

# Carga del nivel probabilístico del embalse
sheet_name='DemandaUPME_2044'
df_DemMes=pd.read_excel(filepath, header=0,sheet_name=sheet_name)
# df_DemMes=df_DemMes.set_index('Embalse_Sinergox')

In [8]:
df_DemMes

,Fecha,Esc. Medio,IC Superior 95%,IC Inferior 95%,IC Superior 68%,IC Inferior 68%
0,2025-04-01,6907.489142,7067.714032,6747.791688,6995.252774,6819.995742
1,2025-05-01,7160.999343,7326.904954,6995.639869,7251.874605,7070.403893
2,2025-06-01,6940.408197,7101.052420,6780.292791,7028.401519,6852.685813
3,2025-07-01,7166.581411,7402.879065,6931.385049,7296.025088,7037.701972
4,2025-08-01,7191.521672,7427.948539,6956.196699,7321.036132,7062.571759
...,...,...,...,...,...,...
172,2039-08-01,10127.926066,11920.643984,8380.940175,11111.015432,9168.251767
173,2039-09-01,9986.900923,11750.590091,8268.203263,10954.071524,9042.766239
174,2039-10-01,10211.765041,12023.523267,8446.283198,11204.753642,9242.469942
175,2039-11-01,10098.690480,11891.285889,8351.881994,11081.176322,9139.647536


In [9]:
df_Factor_typo_d

,Mes,day_osf,Periodo,FactorDay,FactorHour,F_Car,F_ATLANTIC,F_BOLIVAR,F_CERROMAT,F_CORDOSUC,F_GCM,F_Interior,F_Caribe2
0,1,1,1,0.033375,0.036893,0.332605,0.087561,0.056893,0.028400,0.067921,0.091830,0.667395,0.236285
1,1,1,2,0.033375,0.035632,0.333581,0.087820,0.057180,0.029082,0.067998,0.091501,0.666419,0.236501
2,1,1,3,0.033375,0.034780,0.332513,0.087589,0.056872,0.029261,0.067358,0.091432,0.667487,0.235894
3,1,1,4,0.033375,0.034370,0.327857,0.086322,0.055937,0.029378,0.066376,0.089844,0.672143,0.232103
4,1,1,5,0.033375,0.034842,0.316212,0.083570,0.053659,0.029008,0.064031,0.085943,0.683788,0.223173
...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,12,3,20,0.030029,0.050142,0.295232,0.074780,0.052793,0.023761,0.057270,0.086627,0.704768,0.214200
860,12,3,21,0.030029,0.049644,0.303132,0.077022,0.053998,0.024077,0.058598,0.089437,0.696868,0.220457
861,12,3,22,0.030029,0.046155,0.310912,0.080176,0.055382,0.025283,0.059036,0.091035,0.689088,0.226594
862,12,3,23,0.030029,0.044078,0.325570,0.084613,0.058265,0.026394,0.061632,0.094666,0.674430,0.237544


In [14]:
from datetime import datetime
import pandas as pd

tipo='MP'
d_YearIni=2026
d_YearFin=2036

df_Factor_typo_d_adjusted = df_Factor_typo_d.copy()
iFactorInterior=-0.0089802
iFactorCaribe=0.00114209
iFactorCaribe2=0.00223202

df_DemMod=pd.DataFrame()

for year in range(d_YearIni,d_YearFin+1):

    df_Factor_typo_d_adjusted['F_Interior'] += iFactorInterior
    df_Factor_typo_d_adjusted[['F_CERROMAT', 'F_CORDOSUC']] += iFactorCaribe
    df_Factor_typo_d_adjusted[['F_ATLANTIC', 'F_BOLIVAR', 'F_GCM']] += iFactorCaribe2

    if year==2025:
        perini=12
        perfin=13
    else:
        perini=1
        perfin=13

    for mes in range(perini,perfin):
        
        fecha_str = f"{year}-{mes:02d}-01"

        if tipo=='MP':
            fechas = pd.date_range(start=fecha_str, periods=pd.Period(fecha_str).days_in_month, freq='D')
        elif tipo=='LP':
            # Crear fechas para los primeros 5 días del mes
            fechas = pd.date_range(start=fecha_str, periods=5, freq='D')
            # Crear columna 'DiaTipo' según la posición del día en el mes
            dia_tipo_map = {0: 1, 1: 2, 2: 3, 3: 1, 4: 1}
            df_diatipo = pd.DataFrame({'Fecha': fechas, 'day_osf': [dia_tipo_map[i] for i in range(len(fechas))]})

        # Crear dataframe con 24 periodos para cada día

        df_mes = pd.DataFrame([
            {'Fecha': fecha, 'Periodo': periodo}
            for fecha in fechas
            for periodo in range(1, 25)
        ])
        
        if tipo=='MP':
            if tipod=='day_osf':
                df_mes['day_osf']=df_mes.apply(lambda row: typedays(row,tipo='DayType'),axis=1)
            else:
                df_mes['day_w']=df_mes.apply(lambda row: typedays(row,tipo='WeekDay'),axis=1)
        elif tipo=='LP':
            df_mes = df_mes.merge(df_diatipo, on='Fecha', how='left')
        
        d_date = datetime.strptime(fecha_str, '%Y-%m-%d')
        df_DemDia=df_DemMes[(df_DemMes['Fecha']==d_date)]['Esc. Medio']
        valor = df_DemDia.values[0]
        df_mes['ValorMes']=valor
        
        df_mes['Mes'] = df_mes['Fecha'].dt.month

        if tipod=='day_osf':

            df_mes=df_mes.merge(df_Factor_typo_d_adjusted,left_on=['Mes','day_osf','Periodo'],
                                right_on=['Mes','day_osf','Periodo'], how='left')[['Fecha','Mes','day_osf','Periodo','ValorMes','FactorDay','FactorHour','F_Car','F_Interior','F_ATLANTIC', 'F_BOLIVAR','F_CERROMAT', 'F_CORDOSUC', 'F_GCM']]
        else:
            df_mes=df_mes.merge(df_Factor_typo_d_adjusted,left_on=['Mes','day_w','Periodo'],
                                right_on=['Mes','day_w','Periodo'], how='left')[['Fecha','Mes','day_w','Periodo','ValorMes','FactorDay','FactorHour','F_Car','F_Interior','F_ATLANTIC', 'F_BOLIVAR','F_CERROMAT', 'F_CORDOSUC', 'F_GCM']]
        
        df_mes['SubAntioquia'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_Interior']
        df_mes['SubAtlantico'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_ATLANTIC']
        df_mes['SubBolivar'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_BOLIVAR']
        df_mes['SubGCM'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_GCM']
        df_mes['SubCordoba-Sucre'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_CORDOSUC']
        df_mes['SubCerromatoso'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_CERROMAT']


        df_mes=df_mes[['Fecha','Periodo','SubAntioquia','SubAtlantico','SubBolivar','SubGCM','SubCordoba-Sucre','SubCerromatoso']]

        df_mes = df_mes.melt(id_vars=['Fecha', 'Periodo'], var_name='Subarea', value_name='Valor')
        df_mes['Fecha'] = df_mes['Fecha'].dt.date

        df_DemMod=pd.concat([df_DemMod,df_mes],axis=0)


df_DemMod.to_csv(sRutaPrint.joinpath('DatosDemanda.csv'),index=False)

In [15]:
df_Factor_typo_d_adjusted['F_Total'] = df_Factor_typo_d_adjusted['F_ATLANTIC'] + df_Factor_typo_d_adjusted['F_BOLIVAR'] + df_Factor_typo_d_adjusted['F_GCM']+df_Factor_typo_d_adjusted['F_CERROMAT'] +df_Factor_typo_d_adjusted['F_CORDOSUC']+df_Factor_typo_d_adjusted['F_Interior']

In [16]:
df_Factor_typo_d_adjusted

,Mes,day_osf,Periodo,FactorDay,FactorHour,F_Car,F_ATLANTIC,F_BOLIVAR,F_CERROMAT,F_CORDOSUC,F_GCM,F_Interior,F_Caribe2,F_Total
0,1,1,1,0.033375,0.036893,0.332605,0.112113,0.081446,0.040963,0.080484,0.116383,0.568613,0.236285,1.0
1,1,1,2,0.033375,0.035632,0.333581,0.112372,0.081732,0.041645,0.080561,0.116054,0.567637,0.236501,1.0
2,1,1,3,0.033375,0.034780,0.332513,0.112141,0.081425,0.041824,0.079921,0.115984,0.568705,0.235894,1.0
3,1,1,4,0.033375,0.034370,0.327857,0.110875,0.080489,0.041941,0.078939,0.114396,0.573361,0.232103,1.0
4,1,1,5,0.033375,0.034842,0.316212,0.108123,0.078211,0.041571,0.076594,0.110495,0.585006,0.223173,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,12,3,20,0.030029,0.050142,0.295232,0.099332,0.077346,0.036324,0.069833,0.111179,0.605986,0.214200,1.0
860,12,3,21,0.030029,0.049644,0.303132,0.101574,0.078550,0.036640,0.071161,0.113989,0.598086,0.220457,1.0
861,12,3,22,0.030029,0.046155,0.310912,0.104728,0.079934,0.037846,0.071599,0.115587,0.590305,0.226594,1.0
862,12,3,23,0.030029,0.044078,0.325570,0.109165,0.082817,0.038957,0.074195,0.119219,0.575647,0.237544,1.0


In [74]:
df_DemDia

140    10762.815428
Name: IC Superior 95%, dtype: float64

In [75]:

df_mes

,Fecha,Periodo,Subarea,Valor
0,2036-12-01,1,SubAntioquia,9131.946965
1,2036-12-01,2,SubAntioquia,8799.564511
2,2036-12-01,3,SubAntioquia,8601.226111
3,2036-12-01,4,SubAntioquia,8549.894776
4,2036-12-01,5,SubAntioquia,8802.574603
...,...,...,...,...
4459,2036-12-31,20,SubCerromatoso,405.351829
4460,2036-12-31,21,SubCerromatoso,405.951515
4461,2036-12-31,22,SubCerromatoso,395.637178
4462,2036-12-31,23,SubCerromatoso,394.578464


In [76]:
from pydataxm import *                           #Se realiza la importación de las librerias necesarias para ejecutar                        
from pydataxm.pydataxm import ReadDB as apiXM 

s_path=Path.cwd()
s_prt_path=s_path.parent

objetoAPI = pydataxm.ReadDB()                    #Se almacena el servicio en el nombre objetoAPI
df =objetoAPI.get_collections()   

FechaIni=dt.date(2024, 9, 1)
FechaFin=dt.date(2024, 11, 30)

In [77]:
# Unit kWh
df_Dem= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "DemaSIN",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Sistema",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta

### Cálcular día

In [78]:
# Rango de análisis

s_FechaIni='2024-11-25'
s_FechaFin='2024-12-1'

#Cambiar a formato fecha
d_FechaIni=dt.datetime.strptime(s_FechaIni,'%Y-%m-%d')
d_FechaFin=dt.datetime.strptime(s_FechaFin,'%Y-%m-%d')

# Create a DataFrame of dates
df_dates = pd.DataFrame(pd.date_range(start=s_FechaIni, end=s_FechaFin, freq='D')  , columns=["Fecha"])

df_dates['Mes']=df_dates['Fecha'].dt.month

# Definir tipo de día
df_dates['day_osf']=df_dates.apply(lambda row: typedays(row,tipo='DayType'),axis=1)
# Definir día de la semana
df_dates['day_w']=df_dates.apply(lambda row: typedays(row,tipo='WeekDay'),axis=1)

In [79]:
df_dates

,Fecha,Mes,day_osf,day_w
0,2024-11-25,11,1,0
1,2024-11-26,11,1,1
2,2024-11-27,11,1,2
3,2024-11-28,11,1,3
4,2024-11-29,11,1,4
5,2024-11-30,11,2,5
6,2024-12-01,12,3,6


In [80]:
# Definir el valor de energía del mes de la UMPE
ValorMes=10000

periods = [i for i in range(1,25)]  # Create 24 period labels

df_DemRes=pd.DataFrame()

for ind in df_dates.index:
    date=df_dates.loc[ind,'Fecha']

    Mes=date.month
    day_osf=df_dates.loc[ind,'day_osf']
    day_w=df_dates.loc[ind,'day_w']


    # Validar si esciste el día de la semana para el festivo que se está presentando, sino acercar al festivo que se tenga
    # Si es semana validar si hay otro en semana, sino tomar el domingo
    if day_osf==3:
        a=1
    

    df_Fac=df_Factor[(df_Factor.Mes==Mes) & (df_Factor.day_osf==day_osf) & (df_Factor.day_w==day_w)]

    df_Axu=pd.DataFrame({'Periodo': periods})
    df_Axu['Fecha']=date


df_Axu
    # print(date)

,Periodo,Fecha
0,1,2024-12-01
1,2,2024-12-01
2,3,2024-12-01
3,4,2024-12-01
4,5,2024-12-01
5,6,2024-12-01
6,7,2024-12-01
7,8,2024-12-01
8,9,2024-12-01
9,10,2024-12-01


### Factores con demanda real

In [81]:
from pydataxm import *                           #Se realiza la importación de las librerias necesarias para ejecutar                        
from pydataxm.pydataxm import ReadDB as apiXM 
import datetime as dt                            
import pandas as pd
from pathlib import Path
import os

In [82]:
s_path=Path.cwd()
s_prt_path=s_path.parent
s_pathPrint=s_path.joinpath('Print')

objetoAPI = pydataxm.ReadDB()                    #Se almacena el servicio en el nombre objetoAPI
df =objetoAPI.get_collections()  

FechaIni=dt.date(2023, 10, 1)
FechaFin=dt.date(2025, 12, 31)

In [83]:
df_DemRealIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "DemaReal",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Sistema",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta

df_DemSINIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "DemaSIN",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Sistema",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta
df_DemSINIni.rename(columns={'Value':'DemSIN'},inplace=True)
df_DemSINIni['DemSIN']=round(df_DemSINIni['DemSIN']/1e6,2)

In [84]:
df_DemReal=df_DemRealIni.copy()

l_per=[]
for i in range(1,25):
    if i<10:
        df_DemReal.rename(columns={'Values_Hour0' + str(i): i}, inplace=True)
        l_per.append(i)
    else:
        df_DemReal.rename(columns={'Values_Hour' + str(i): i}, inplace=True)
        l_per.append(i)

# df_DemReal=df_DemReal.groupby(['Date'])[l_per].mean()
# Get mean of numeric columns
df_DemReal_mean = pd.DataFrame(df_DemReal[l_per].mean())
Total=df_DemReal_mean[[0]].sum()
df_DemReal_mean['Factor'] = df_DemReal_mean[0] / Total[0]
df_DemReal_mean.to_csv(s_pathPrint.joinpath('Factores.csv'))

# df_DemReal['TotalDem']=0
# for col in l_per:
#     df_DemReal['TotalDem']=df_DemReal['TotalDem']+df_DemReal[col]

In [85]:
# Definir los periodos en una columna
# df_DemSIN = pd.melt(df_DemSIN, id_vars=['Fecha'], var_name='Periodo', value_name='DemMW')
# # df_dataP['Precio']=df_dataP['Precio']/1000
df_DemSIN=df_DemSINIni.copy()

df_DemSIN['Yaer']=df_DemSIN['Date'].dt.year
df_DemSIN['Mes']=df_DemSIN['Date'].dt.month
df_DemSIN['Dia']=df_DemSIN['Date'].dt.day

df_DemSIN['Year-Mes']=df_DemSIN['Yaer'].astype(str) + '-' + df_DemSIN['Mes'].astype(str)

df_DemSIN=df_DemSIN.groupby(['Year-Mes'])[['DemSIN']].sum()


In [86]:
df_DemSIN.to_csv('DemandaSIN.csv')